In [5]:
from modules.statistical import *

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:20<00:00,  3.38s/it]


In [6]:
import numpy as np

In [3]:
layers = list(range(1,14))
layers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [4]:
alphas = [round(i,1) for i in np.arange(0.1,1.0,0.1)]
alphas.append(0.85)
#alphas = sorted(list(set(alphas)-set([0.2, 0.4])))
alphas

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.85]

# generation du tableau de comparaison de qualité de descripteurs dans la processus de selection

generate_g_b_impact_table(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    _type='qualitative',
    alphas=alphas
)

# Generation de tableau de comparaison de l'impact metriques des descripteurs

generate_report_tables_v3_2(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    glo=True,
    per=True,
    gap=True,
    logics=['GLO','PER'],#,'PER','GAP'
    approachs=['MlC','MCA'],
    configs=['MX','CX','CY','CXY'],#'MX',
    metrics=['accuracy','precision1','precision0','financial-cost'],#'accuracy','f1-score','precision1','recall1','f1-score1','precision0','recall0','f1-score0','financial-cost'
    metrics1=['precision1','financial-cost'],#'accuracy','f1-score','precision1','recall1','f1-score1','precision0','recall0','f1-score0','financial-cost'
    _type='qualitative',
    alphas=alphas,
    result_=[1],
    total_digits= None,
    decimal_digits= 0
)

a = {'precision1': {0.1: 0.5, 0.2: 0.6, 0.3: 1.0, 0.4: 0.7, 0.5: 0.7, 0.6: 0.5, 0.7: 0.5, 0.8: 0.8, 0.85: 0.8, 0.9: 6.8999999999999995}, 'financial-cost': {0.1: 4.0, 0.2: 2.7, 0.3: 1.1, 0.4: 1.5, 0.5: 3.5, 0.6: 1.5999999999999999, 0.7: 3.2, 0.8: 2.6, 0.85: 2.8000000000000003, 0.9: 2.4}}
a

# Generation de bar chart pour l'evaluation des descripteurs et du tableau recapitulatifs

generate_descriptor_ranking(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    _type='qualitative',
    alphas=alphas
)

# Proc Selection Analysis

In [5]:
import numpy as np

def vector_matching_precision(v1, v2, tolerance=0):
    """
    Calcule la précision de correspondance entre deux vecteurs.
    
    :param v1: Premier vecteur
    :param v2: Deuxième vecteur
    :param tolerance: Tolérance pour considérer deux valeurs comme correspondantes
    :return: Pourcentage de correspondance entre les vecteurs
    """
    if len(v1) != len(v2):
        raise ValueError("Les vecteurs doivent avoir la même longueur")
    
    v1, v2 = np.array(v1), np.array(v2)
    matches = np.abs(v1 - v2) <= tolerance
    precision = np.mean(matches) * 100
    
    return precision

In [6]:
def print_compare_feature_selection_protocole(
    store,
    output_path
):
    # add the resize box to ensure the scale of the table will be contain's inside the width space avalable.
    # start setting up the tabular dimensions setting
    table_header = """
        %\\begin{sidewaystable}
        \\resizebox{\\textwidth}{!}{

        \\begin{tabular}{|c|c|"""+("c|"*10)
    # setup information columns headears
    nbMCol =  10
    # add col for total results
    table_header+= "} "
    # add separator clines
    nb_cols = (2+nbMCol)
    table_header+= " \\cline{1-"+str(nb_cols)+"}" # corresponding to the number of columns

    # build the first line: metrics' line
    lines = ''
    # add the blank block
    lines += """
    \\multicolumn{2}{|c|}{}"""


    datasets = list(store.keys())
    methods = list(store[datasets[0]].keys())
    alphas = sorted(list({el for data in datasets for el in store[data][methods[0]].keys() }))
    # add alpha for metric
    for alpha in alphas:
        lines+= f" & {alpha}"
    # add the total name
    lines+= " \\\\ "
    lines+= " \\cline{1-"+str(nb_cols)+"""}
    """
    for folder in datasets:
        # fetch on model
        lines+= """
            \\multirow{3}{*}{"""+folder+"""}"""
        for mi, meth in enumerate(methods): 
            lines+= f""" & {meth}"""
            for ai, alpha in enumerate(alphas): # MlC, MCA
                lines+= f""" & {store[folder][meth][alpha] }""" if alpha in list(store[folder][meth].keys()) else " & "
            lines+= ("""\\\\ """+ """ \\cline{2-"""+str(nb_cols)+"""}

                    """) if mi != len(methods)-1 else ("""\\\\ """+ """ \\cline{1-"""+str(nb_cols)+"""}

                    """)
                

    lines+= """

    \\end{tabular}
    }
    %\\end{sidewaystable}"""

    table = table_header + lines
    create_domain(f"{output_path}/descriptComp/alpha/mlnGrowth1")
    filename1 = f"{output_path}/descriptComp/alpha/mlnGrowth1/mlnGrowth1.tex"
    _file = open(filename1, "w")
    _file.write(header+"""
                \\begin{table}[H]
                \\centering
                """+table+"""
                \\caption{default}
                \\label{default}
                \\end{table}"""+footer)
    _file.close()

In [7]:
records = joblib.load('./result_lts/best.tex')

# result structure
resultDict = {
    'dataset':[],
    'alpha': [],
    # 'QuartileThreshold':[],
    'elbowThreshold':[],
    'cumulative_difference_threshold':[],
    # 'variance_explained_threshold':[],
    'realThreshold':[]
}
res = {}
getTheBestAcc = lambda store, k: round(max([acc for layer,_,acc in store if k == layer]),4)

# walk on the datasets
for dataset in records.keys():
    # walk on alphas
    res[dataset] = {key:{} for key in ['CUSUM','Elbow','réel']}
    for alpha in records[dataset].keys():
        if len(records[dataset][alpha]['predicted_best_k']) != 0:
            resultDict['dataset'].append(dataset)
            resultDict['alpha'].append(alpha)
            # resultDict['QuartileThreshold'].append(getTheBestAcc(records[dataset][alpha]['list'],records[dataset][alpha]['predicted_best_k'][0]))
            elb = elbow_method(list(records[dataset][alpha]['accuracies'].values()))
            elb = elb if elb < max([layer for layer,_,_ in records[dataset][alpha]['list']]) else max([layer for layer,_,_ in records[dataset][alpha]['list']])
            resultDict['elbowThreshold'].append(getTheBestAcc(records[dataset][alpha]['list'],elb))
            res[dataset]['Elbow'][alpha] = getTheBestAcc(records[dataset][alpha]['list'],elb)
            cum = cumulative_difference_threshold(list(records[dataset][alpha]['accuracies'].values()))
            cum = cum if cum < max([layer for layer,_,_ in records[dataset][alpha]['list']]) else max([layer for layer,_,_ in records[dataset][alpha]['list']]) 
            resultDict['cumulative_difference_threshold'].append(getTheBestAcc(records[dataset][alpha]['list'],cum))
            res[dataset]['CUSUM'][alpha] = getTheBestAcc(records[dataset][alpha]['list'],cum)
            # resultDict['variance_explained_threshold'].append(getTheBestAcc(records[dataset][alpha]['list'],variance_explained_threshold(list(records[dataset][alpha]['accuracies'].values()))))
            resultDict['realThreshold'].append(round(max([acc for _,_,acc in records[dataset][alpha]['list']]),4) )
            res[dataset]['réel'][alpha] = round(max([acc for _,_,acc in records[dataset][alpha]['list']]),4)

dat = pd.DataFrame(resultDict)
print_compare_feature_selection_protocole(res,'result_lts')

Directory 'result_lts/descriptComp/alpha/mlnGrowth1' created successfully.


/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/modules/pipeline.py:5265: RuntimeWarning: invalid value encountered in scalar divide
  if cumulative_diff / total_diff >= threshold_percent:


In [8]:
# print(f"\nPrécision QuartileThreshold avec tolérance 0: {vector_matching_precision(resultDict['QuartileThreshold'], resultDict['realThreshold']):.2f}%")
print(f"\nPrécision elbowThreshold avec tolérance 1%: {vector_matching_precision(resultDict['elbowThreshold'], resultDict['realThreshold'],0.01):.2f}%")
print(f"\nPrécision cumulative_difference_threshold avec tolérance 1%: {vector_matching_precision(resultDict['cumulative_difference_threshold'], resultDict['realThreshold'], 0.01):.2f}%")
# print(f"\nPrécision variance_explained_threshold avec tolérance 0: {vector_matching_precision(resultDict['variance_explained_threshold'], resultDict['realThreshold']):.2f}%")


Précision elbowThreshold avec tolérance 1%: 70.27%

Précision cumulative_difference_threshold avec tolérance 1%: 64.86%


In [1]:
import numpy as np

# Données réelles (ground truth)
real_values = {
    "LD4": [0.9737, 0.9699, 0.9699, 0.9699, 0.9737, 0.9699, 0.9699, 0.9699, 0.9699],
    "GER": [0.795, 0.805, 0.805, 0.8, 0.8, 0.795, 0.79, 0.795, 0.8],
    "CRD": [0.9123, 0.9079, 0.9079, 0.9079, 0.9123, 0.9035, 0.9035, 0.9123, 0.9079, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

# Données estimées (Elbow & CUSUM)
elbow_values = {
    "LD4": [0.9699, 0.9699, 0.9662, 0.9662, 0.9699, 0.9662, 0.9699, 0.9662, 0.9699],
    "GER": [0.78, 0.79, 0.78, 0.79, 0.785, 0.775, 0.765, 0.79, 0.78],
    "CRD": [0.9079, 0.8991, 0.9035, 0.9079, 0.8991, 0.9035, 0.9035, 0.8991, 0.9035, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

cusum_values = {
    "LD4": [0.9699, 0.9699, 0.9662, 0.9662, 0.9699, 0.9662, 0.9699, 0.9662, 0.9699],
    "GER": [0.78, 0.77, 0.78, 0.745, 0.77, 0.77, 0.77, 0.76, 0.765],
    "CRD": [0.9079, 0.8991, 0.9079, 0.9079, 0.8991, 0.9035, 0.9035, 0.8991, 0.9035, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

# Tolérances à tester
tolerances = [0.01, 0.02, 0.03, 0.04, 0.05]

# Fonction pour calculer la précision
def compute_precision(estimated_values, real_values, tolerance):
    precisions = {}
    for dataset in real_values:
        real = np.array(real_values[dataset])
        estimated = np.array(estimated_values[dataset])
        precision = np.mean(np.abs(real - estimated) <= tolerance) * 100
        precisions[dataset] = precision
    return precisions

# Calcul des précisions pour chaque tolérance
elbow_results = {tol: compute_precision(elbow_values, real_values, tol) for tol in tolerances}
cusum_results = {tol: compute_precision(cusum_values, real_values, tol) for tol in tolerances}

# Affichage des résultats
print("Précision Elbow:")
for tol in tolerances:
    print(f"Tolérance {tol*100:.0f}% :", elbow_results[tol])

print("\nPrécision CUSUM:")
for tol in tolerances:
    print(f"Tolérance {tol*100:.0f}% :", cusum_results[tol])


Précision Elbow:
Tolérance 1% : {'LD4': 100.0, 'GER': 11.11111111111111, 'CRD': 80.0, 'LDD': 100.0}
Tolérance 2% : {'LD4': 100.0, 'GER': 55.55555555555556, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 3% : {'LD4': 100.0, 'GER': 100.0, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 4% : {'LD4': 100.0, 'GER': 100.0, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 5% : {'LD4': 100.0, 'GER': 100.0, 'CRD': 100.0, 'LDD': 100.0}

Précision CUSUM:
Tolérance 1% : {'LD4': 100.0, 'GER': 0.0, 'CRD': 80.0, 'LDD': 100.0}
Tolérance 2% : {'LD4': 100.0, 'GER': 11.11111111111111, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 3% : {'LD4': 100.0, 'GER': 44.44444444444444, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 4% : {'LD4': 100.0, 'GER': 88.88888888888889, 'CRD': 100.0, 'LDD': 100.0}
Tolérance 5% : {'LD4': 100.0, 'GER': 88.88888888888889, 'CRD': 100.0, 'LDD': 100.0}


In [2]:
import numpy as np

# Données réelles (ground truth)
real_values = {
    "LD4": [0.9737, 0.9699, 0.9699, 0.9699, 0.9737, 0.9699, 0.9699, 0.9699, 0.9699],
    "GER": [0.795, 0.805, 0.805, 0.8, 0.8, 0.795, 0.79, 0.795, 0.8],
    "CRD": [0.9123, 0.9079, 0.9079, 0.9079, 0.9123, 0.9035, 0.9035, 0.9123, 0.9079, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

# Données estimées (Elbow & CUSUM)
elbow_values = {
    "LD4": [0.9699, 0.9699, 0.9662, 0.9662, 0.9699, 0.9662, 0.9699, 0.9662, 0.9699],
    "GER": [0.78, 0.79, 0.78, 0.79, 0.785, 0.775, 0.765, 0.79, 0.78],
    "CRD": [0.9079, 0.8991, 0.9035, 0.9079, 0.8991, 0.9035, 0.9035, 0.8991, 0.9035, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

cusum_values = {
    "LD4": [0.9699, 0.9699, 0.9662, 0.9662, 0.9699, 0.9662, 0.9699, 0.9662, 0.9699],
    "GER": [0.78, 0.77, 0.78, 0.745, 0.77, 0.77, 0.77, 0.76, 0.765],
    "CRD": [0.9079, 0.8991, 0.9079, 0.9079, 0.8991, 0.9035, 0.9035, 0.8991, 0.9035, 0.9123],
    "LDD": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

# Tolérances à tester
tolerances = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05]

# Fonction pour calculer la précision
def compute_precision(estimated_values, real_values, tolerance):
    precisions = {}
    for dataset in real_values:
        real = np.array(real_values[dataset])
        estimated = np.array(estimated_values[dataset])
        precision = np.mean(np.abs(real - estimated) <= tolerance) * 100
        precisions[dataset] = precision
    return precisions

# Calcul des précisions pour chaque tolérance
elbow_results = {tol: compute_precision(elbow_values, real_values, tol) for tol in tolerances}
cusum_results = {tol: compute_precision(cusum_values, real_values, tol) for tol in tolerances}

# Génération du tableau LaTeX
latex_table = """
\\begin{tabular}{|c|c|c|c|c||c|c|c|c|}
    \\hline
    \\multirow{3}{*}{} & \\multicolumn{8}{|c|}{Estimation du meilleur k} \\\\ \\cline{2-9}
    & \\multicolumn{4}{|c|}{Elbow} & \\multicolumn{4}{|c|}{CUSUM} \\\\ \\cline{2-9}
    & LD4 & GER & CRD & LDD & LD4 & GER & CRD & LDD \\\\ \\hline
"""

for tol in tolerances:
    row = f"    Précision (diff.{int(tol*100)}\\%) & "
    row += " & ".join([f"{elbow_results[tol][dataset]:.1f}" for dataset in ["LD4", "GER", "CRD", "LDD"]])
    row += " & "
    row += " & ".join([f"{cusum_results[tol][dataset]:.1f}" for dataset in ["LD4", "GER", "CRD", "LDD"]])
    row += " \\\\ \\hline\n"
    latex_table += row

latex_table += "\\end{tabular}"

# Affichage du tableau LaTeX
print(latex_table)



\begin{tabular}{|c|c|c|c|c||c|c|c|c|}
    \hline
    \multirow{3}{*}{} & \multicolumn{8}{|c|}{Estimation du meilleur k} \\ \cline{2-9}
    & \multicolumn{4}{|c|}{Elbow} & \multicolumn{4}{|c|}{CUSUM} \\ \cline{2-9}
    & LD4 & GER & CRD & LDD & LD4 & GER & CRD & LDD \\ \hline
    Précision (diff.0\%) & 33.3 & 0.0 & 40.0 & 100.0 & 33.3 & 0.0 & 50.0 & 100.0 \\ \hline
    Précision (diff.1\%) & 100.0 & 11.1 & 80.0 & 100.0 & 100.0 & 0.0 & 80.0 & 100.0 \\ \hline
    Précision (diff.2\%) & 100.0 & 55.6 & 100.0 & 100.0 & 100.0 & 11.1 & 100.0 & 100.0 \\ \hline
    Précision (diff.3\%) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 44.4 & 100.0 & 100.0 \\ \hline
    Précision (diff.4\%) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 88.9 & 100.0 & 100.0 \\ \hline
    Précision (diff.5\%) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 88.9 & 100.0 & 100.0 \\ \hline
\end{tabular}


In [10]:
records = joblib.load('./result_lts_140225/macro_metrics.tex')

In [28]:
results = joblib.load('./result_lts_140225/macro_metrics.tex')
# Sélectionner les meilleures valeurs pour chaque catégorie
for config, metric_data in results.items():
    for metric, cat_data in metric_data.items():
        for cat, values in cat_data.items():
            if values:
                results[config][metric][cat] = max(values, key=lambda x: x[0])

# # Génération du tableau LaTeX
# latex_table = """
# \begin{tabular}{|c|c|c|c|c||c|c|c|c||c|c|c|c|}
#     \hline
#     \multirow{3}{*}{} & \multicolumn{4}{|c|}{Acc} & \multicolumn{4}{|c|}{F1-score} & \multicolumn{4}{|c|}{Cost} \\
#     \cline{2-13}
#     & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD \\
#     \hline
# """

# for config in results.keys():
#     row = f"    {config} "
#     for metric in ["Acc", "F1-score", "Cost"]:
#         for cat in ["CRD", "GER", "LD4", "LDD"]:
#             value, algo = results[config][metric].get(cat, (0, "N/A"))
#             row += f" & {value} | {algo}"
#     row += " \\ \hline\n"
#     latex_table += row

# latex_table += "\end{tabular}"

# # Affichage du tableau LaTeX
# print(latex_table)


In [53]:
import numpy as np

# Structure des données extraites des 4 tableaux
# results = {
#     "MlC_GLO_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_GLO_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_PER_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_PER_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_PER_CY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_PER_CXY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_GAP_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_GAP_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_GAP_CY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MlC_GAP_CXY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GLO_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GLO_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_PER_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_PER_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_PER_CY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_PER_CXY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GAP_MX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GAP_CX": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GAP_CY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                    "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}},
    
#     "MCA_GAP_CXY": {"Acc": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "F1-score": {"CRD": [], "GER": [], "LD4": [], "LDD": []},
#                     "Cost": {"CRD": [], "GER": [], "LD4": [], "LDD": []}}
# }

# Initialiser un compteur pour le total des meilleures valeurs par configuration
total_best_counts = {config: 0 for config in results.keys()}
# Trouver les meilleures améliorations par colonne
metrics = ["Acc", "F1-score", "Cost"]
categories = ["CRD", "GER", "LD4", "LDD"]
best_values = {metric: {cat: (-np.inf, "") for cat in categories} for metric in metrics}

for config, metric_data in results.items():
    for metric, cat_data in metric_data.items():
        for cat, (value, algo) in cat_data.items():
             if value > best_values[metric][cat][0]:
                best_values[metric][cat] = (value, algo)

# Compter le nombre de fois qu'une ligne détient les meilleurs résultats
for config, metric_data in results.items():
    for metric in metrics:
        for cat in categories:
            if round(results[config][metric][cat][0],1) == round(best_values[metric][cat][0],1):
                total_best_counts[config] += 1

# Trier les configurations par total décroissant
sorted_totals = sorted(total_best_counts.items(), key=lambda x: x[1], reverse=True)
best_total, second_best_total, third_best_total = sorted_totals[0][1], sorted_totals[1][1], sorted_totals[2][1]

# Génération du tableau LaTeX
latex_table = """
\\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}
    \\hline
    \\multirow{3}{*}{} & \\multicolumn{4}{|c|}{Acc} & \\multicolumn{4}{|c|}{F1-score} & \\multicolumn{4}{|c|}{Cost}  & \\multirow{3}{*}{Total}\\\\
    \\cline{2-13}
    & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD & \\\\
    \\hline
"""

for config in results.keys():
    row=""
    total_value = total_best_counts[config]
    if total_value == best_total:
        row += "\\rowcolor{green}"
    elif total_value == second_best_total:
        row += "\\rowcolor{cyan}"
    elif total_value == third_best_total:
        row += "\\rowcolor{lightgray}"
    row += config.replace('_','\\_')
    for metric in metrics:
        for cat in categories:
            value, algo = results[config][metric][cat]
            best_value, _ = best_values[metric][cat]
            formatted_value = f"\\textbf{{{value:.1f}}}" if round(value,1) == round(best_value,1) else f"{value:.1f}"
            row += f" & {formatted_value} ({algo})"
    # Formater la colonne Total
    
    # if total_value == best_total:
    #     formatted_total = "\\textbf{\\color{blue}"+str(total_value)+"}"
    # elif total_value == second_best_total:
    #     formatted_total = "\\underline{\\color{green}"+str(total_value)+"}"
    # elif total_value == third_best_total:
    #     formatted_total = "\\textit{\\color{red}"+str(total_value)+"}"
    # else:
    formatted_total = f"{total_value}"
    
    row += f" & {formatted_total} \\\\ \\hline\n"
    latex_table += row

latex_table += "\end{tabular}"

# Affichage du tableau LaTeX
print(latex_table)



\begin{tabular}{|c|c|c|c|c||c|c|c|c|c|c|c|c|c|}
    \hline
    \multirow{3}{*}{} & \multicolumn{4}{|c|}{Acc} & \multicolumn{4}{|c|}{F1-score} & \multicolumn{4}{|c|}{Cost}  & \multirow{3}{*}{Total}\\
    \cline{2-13}
    & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD & CRD & GER & LD4 & LDD & \\
    \hline
MlC\_GLO\_MX & 3.0 (RF) & 5.5 (RF) & \textbf{2.4} (DT) & 0.0 (LR) & 2.3 (XGB) & 8.1 (RF) & 8.3 (LR) & 0.0 (LR) & 20.8 (RF) & \textbf{99.9} (LDA) & 87.0 (LDA) & \textbf{99.6} (LDA) & 3 \\ \hline
MlC\_GLO\_CX & 0.5 (LR) & 3.4 (RF) & 2.0 (DT) & 0.4 (LR) & -0.9 (SVM) & 1.0 (LR) & 8.3 (LR) & 0.7 (LR) & 17.3 (LDA) & \textbf{99.9} (RF) & 88.7 (RF) & \textbf{99.6} (LDA) & 2 \\ \hline
MlC\_PER\_MX & \textbf{5.8} (DT) & 6.9 (RF) & 0.4 (RF) & 0.0 (LR) & 2.7 (RF) & 11.5 (RF) & 15.3 (DT) & 0.0 (LR) & 28.9 (RF) & \textbf{99.9} (LDA) & 87.0 (LDA) & \textbf{99.6} (LDA) & 3 \\ \hline
\rowcolor{green}MlC\_PER\_CX & 1.6 (DT) & \textbf{9.7} (RF) & \textbf{2.4} (DT) & \textbf{0.9} (LR) & 3.0 (LR) & 2.3 